In [1]:
import re
import urllib.request 
from urllib.request import Request, urlopen
from IPython.core.display import display, HTML

import pandas as pd

In [2]:
print("Hellooo??")

Hellooo??


In [12]:
#!pip install --upgrade protobuf
#!pip install  gspread
#!pip install  oauth2client


In [3]:
import io 
import csv
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import numpy as np

import requests


import datetime
from datetime import date

In [4]:
#from urllib.request import Request, urlopen

req = Request('https://eeaonline.eea.state.ma.us/DPU/Fileroom/filings/bydate', headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

#print(webpage)

In [5]:

from datetime import timedelta


today = date.today()

today = pd.to_datetime(date.today())

In [6]:
start_date = today - timedelta(days=7)
end_date = today


start = start_date.strftime("%m/%d/%Y")
end = end_date.strftime("%m/%d/%Y")



In [7]:
#ok clicked on network to find link, see if this works


test_link = 'https://eeaonline.eea.state.ma.us/DPU/Fileroom//Filings/GetByFilingTimeFrame/?standAlone=true&start=%s&end=%s' % (start, end)
req = Request(test_link, headers={'User-Agent': 'Mozilla/5.0'})
webpage1 = urlopen(req).read().decode("utf-8")

print(test_link)

#print(webpage1)




https://eeaonline.eea.state.ma.us/DPU/Fileroom//Filings/GetByFilingTimeFrame/?standAlone=true&start=11/29/2019&end=12/06/2019


In [9]:
#this has the DATE and the FILER and the DOCKET and Initial Filing and DESCRIPTION
#update: added the \r so its removed from the description for clean purposes
#update2: the \r will make it so some descriptions are not caputred due to an extra space in the description, would rather clean that \r up later
#FINAL Verz 1
results2 = re.findall('<span class="created"><strong>(.*)</strong></span><span>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;</span>\s*.*</div>\s*.*<div class="right">\s*.*<strong><label for="Filer">Filer</label>: </strong><span class="filer">(.*)</span><span>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;</span>\s*.*;">(.*)</a>.*</span>\s*<br />\s*.*<strong><label for="FilingType">Document Type</label>: </strong><span class="filingtype">(Initial\sFiling).*\s*\n*<br />\s*<br />\s*<div class="description">\s*(.*)',webpage1)
#print(results2)

[]


In [10]:
#ok this cathces the DATE and the FILER(AGO ONLY) and the DOCKET and the speciific AAG
#Final Verz 1
#update after user testing:  expand field of AAGs collected
results4 = re.findall('<span class="created"><strong>(.*)</strong></span><span>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;</span>\s*.*</div>\s*.*<div class="right">\s*.*<strong><label for="Filer">Filer</label>: </strong><span class="filer">(Attorney\sGeneral)</span><span>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;</span>\s*.*;">(.*)</a>.*</span>\s*<br />\s*.*<strong><label for="FilingType">Document Type</label>: </strong><span class="filingtype">(Intervention).*\s*\n*<br />\s*<br />\s*<div class="description">\s*.*Appearance of Counsel for\s(\w*\s\w*).*\r',webpage1)
#print(results4)

[('12/02/2019', 'Attorney General', '19-123', 'Intervention', 'Timothy Reppucci')]


In [11]:
#trying to turn results into a data frame
New_Filings = pd.DataFrame(results2, columns = ['Date_Filed' , 'Filer', 'Docket' , 'File Type' , 'Description'])

#cleanup Data

New_Filings['Docket'] = New_Filings['Docket'].str.lower()

display(New_Filings)

,Date_Filed,Filer,Docket,File Type,Description


In [12]:
#trying to clean up the \r
New_Filings['Description'] = New_Filings['Description'].replace('\r', ' ', regex=True)

#maybe add in the duplicate part here

In [13]:
New_AGO=pd.DataFrame(results4, columns = ['Date_Filed' , 'Filer', 'Docket' , 'File Type' , 'AAG on Case'])

#clean data

New_AGO['Docket'] = New_AGO['Docket'].str.lower()

display(New_AGO)

,Date_Filed,Filer,Docket,File Type,AAG on Case
0,12/02/2019,Attorney General,19-123,Intervention,Timothy Reppucci


In [14]:
#ok now working on showing which ones we don't have AAG on

New_AGO = pd.DataFrame.from_records(results4, exclude=['Filer', 'File Type'], columns = ['Date_Intervention' , 'Filer', 'Docket' , 'File Type' , 'AAG'] )
display(New_AGO)

#merger test

Combined_Files_Int = New_Filings.merge(New_AGO, on='Docket', how='outer') 
display(Combined_Files_Int)
Combined_Files_Int.dtypes

,Date_Intervention,Docket,AAG
0,12/02/2019,19-123,Timothy Reppucci


,Date_Filed,Filer,File Type,Description,Date_Intervention,Docket,AAG
0,NaN,NaN,NaN,NaN,12/02/2019,19-123,Timothy Reppucci


Date_Filed           object
Filer                object
File Type            object
Description          object
Date_Intervention    object
Docket               object
AAG                  object
dtype: object

In [15]:
#adding features


Combined_Files_Int['Date_Filed'] = pd.to_datetime(Combined_Files_Int['Date_Filed'])
Combined_Files_Int['Date_Intervention'] = pd.to_datetime(Combined_Files_Int['Date_Intervention'])

Combined_Files_Int['Days Without an AAG'] = today - Combined_Files_Int['Date_Filed']

#want to remove entries wDaysith an AAG

No_AAGs = Combined_Files_Int[Combined_Files_Int.Date_Intervention.isnull()]

No_AAGs = No_AAGs[['Date_Filed', 'Filer', 'Docket', 'File Type', 'Description']].copy()

#creation of table with cases without an AAG

Cases_Without_AAG = No_AAGs

Cases_Without_AAG.drop_duplicates(subset = 'Docket', keep = 'first', inplace = True)

display(Cases_Without_AAG)

,Date_Filed,Filer,Docket,File Type,Description


In [16]:
#Table of Just New AAGs Added

New_AAG1 = Combined_Files_Int[Combined_Files_Int.Date_Intervention.notnull()]

New_AAG1 = New_AAG1[['Date_Filed', 'Filer', 'Docket', 'File Type', 'Description', 'Date_Intervention', 'AAG']].copy()

Cases_With_AAG = New_AAG1

Cases_With_AAG.drop_duplicates(subset = 'Docket', keep = 'first', inplace = True)

display(Cases_With_AAG)

,Date_Filed,Filer,Docket,File Type,Description,Date_Intervention,AAG
0,NaT,NaN,19-123,NaN,NaN,2019-12-02,Timothy Reppucci


In [26]:
#Lets Catch AGO Briefs
#results6 = re.findall('<span class="created"><strong>(.*)</strong></span><span>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;</span>\s*.*</div>\s*.*<div class="right">\s*.*<strong><label for="Filer">Filer</label>: </strong><span class="filer">(Attorney\sGeneral)</span><span>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;</span>\s*.*;">(.*)</a>.*</span>\s*<br />\s*.*<strong><label for="FilingType">Document Type</label>: </strong><span class="filingtype">(Brief).*\s*\n*<br />\s*<br />\s*<div class="description">\s*(.*)',webpage1)
#print(results6)

In [27]:
#New_Briefs = pd.DataFrame(results6, columns = ['Date_Filed' , 'Filer', 'Docket' , 'File Type' , 'Description'])

#New_Briefs['Description'] = New_Briefs['Description'].replace('\r', ' ', regex=True)

#New_Briefs['Docket'] = New_Briefs['Docket'].str.lower()

#display(New_Briefs)

,Date_Filed,Filer,Docket,File Type,Description


In [28]:
#lets capture new orders
#results8 = re.findall('<span class="created"><strong>(.*)</strong></span><span>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;</span>\s*.*</div>\s*.*<div class="right">\s*.*<strong><label for="Filer">Filer</label>: </strong><span class="filer">(DPU)</span><span>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;</span>\s*.*;">(.*)</a>.*</span>\s*<br />\s*.*<strong><label for="FilingType">Document Type</label>: </strong><span class="filingtype">(Order).*\s*\n*<br />\s*<br />\s*<div class="description">\s*.*(ORDER\.).*',webpage1)
#print(results8)

In [29]:
#df for new orders, which we will need to compare to the record of cases we are on

#All_New_Orders = pd.DataFrame(results8, columns = ['Date_Filed' , 'Filer', 'Docket' , 'File Type' , 'Description'])

#All_New_Orders['Description'] = All_New_Orders['Description'].replace('\r', ' ', regex=True)

#All_New_Orders['Docket'] = All_New_Orders['Docket'].str.lower()

#display(All_New_Orders)


,Date_Filed,Filer,Docket,File Type,Description
0,11/27/2019,DPU,19-62,Order,ORDER.
1,11/26/2019,DPU,18-sq-14,Order,ORDER.
2,11/26/2019,DPU,18-sq-13,Order,ORDER.
3,11/26/2019,DPU,18-sq-12,Order,ORDER.
4,11/26/2019,DPU,18-sq-11,Order,ORDER.
5,11/26/2019,DPU,18-sq-10,Order,ORDER.
6,11/26/2019,DPU,19-gc-56,Order,ORDER.
7,11/26/2019,DPU,19-112,Order,ORDER.


In [17]:
# email for your app: owner-596@api-key-first-test.iam.gserviceaccount.com
# this is the email used by your app. See

# location of you key
GKey = r"C:\Users\cwalk\Documents\HOME\keys\API_Key_First_Test-95147982bc22.json"
credentials = ServiceAccountCredentials.from_json_keyfile_name(GKey, ['https://spreadsheets.google.com/feeds'])

# the id for the sheet you want to access
sheet_id = "1YBC5BHOIHPHI_qviJA86TR8JxT7trffKlcSYrjAHOPw"

In [18]:
# import Sheet
def download_sheet(document_key,form_name,skip=None):
    csvfile = r"C:\Users\cwalk\Documents\HOME\raw_data\tmp.csv"
    gc = gspread.authorize(credentials)
    wks = gc.open_by_key(document_key)
    worksheet = wks.worksheet(form_name)    
    list_of_lists = worksheet.get_all_values()
    # uncomment to clean up empty rows and columns
    #worksheet.resize(len(list_of_lists),len(list_of_lists[0]))
    with open(csvfile, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        writer.writerows(list_of_lists)
    return pd.read_csv(csvfile, parse_dates=[0], encoding = "ISO-8859-1", skiprows=skip)


        
# add rows from a dataframe df
def add_rows(document_key,form_name,df):
    gc = gspread.authorize(credentials)
    wks = gc.open_by_key(document_key)
    worksheet = wks.worksheet(form_name)
    df = df.astype(str)
    print("\nAdding...\n")
    for index, row in df.iterrows():
        print(row.tolist())
        worksheet.append_row(row.tolist())
    print("\n")
        
# remove everything but the header from a worksheet
def empty_sheet(document_key,form_name):
    gc = gspread.authorize(credentials)
    wks = gc.open_by_key(document_key)
    worksheet = wks.worksheet(form_name)
    list_of_lists = worksheet.get_all_values()
    worksheet.resize(1,len(list_of_lists[0]))

In [19]:
record = download_sheet(sheet_id,"Record_of_All_Dockets_with_an_AAG")
record['Date_Filed'] = pd.to_datetime(record['Date_Filed'])


#display(record)


In [20]:
add_dockets = Cases_With_AAG[['Date_Filed', 'Docket', 'AAG']]

In [21]:
#display(add_dockets)
#display(record)

add_dockets_uploader = pd.concat([record, add_dockets])
#display(add_dockets_uploader)
#duplicates not gone yet, see next cell

In [22]:
add_dockets_uploader.drop_duplicates(subset = 'Docket', keep = 'first', inplace = True)

#display(add_dockets_uploader)

In [36]:
#analysis of record and orders DFs

#Orders_and_AAGs = All_New_Orders.merge(add_dockets_uploader, on='Docket', how='left') 

#Orders_with_AAGs = Orders_and_AAGs[Orders_and_AAGs.AAG.notnull()]

#Orders_with_AAGs = Orders_with_AAGs[['Date_Filed_x', 'Filer', 'Docket', 'File Type', 'AAG']].copy()

#Orders_with_AAGs.rename(columns={'Date_Filed_x':'Date_Filed'}, inplace=True)

#display(Orders_with_AAGs)

In [37]:
#ok lets recap out results
#to see all Orders for cases with an AAG:  Orders_with_AAGs

#to see all new orders:  All_New_Orders

#to see all new AGO Briefs:  New_Briefs

#to see all Dockets without an AAG:  Cases_Without_AAG

#to see all Dockets with new AAG intervention:  Cases_With_AAG

#to see all Dockets, from start of program launch, with an AAG:  add_dockets_uploader

In [38]:
#display(All_New_Orders)

In [23]:
#Clear out sheet before uploading

empty_sheet(sheet_id,"Record_of_All_Dockets_with_an_AAG")

In [24]:
add_rows(sheet_id,"Record_of_All_Dockets_with_an_AAG",add_dockets_uploader)


Adding...

['2019-11-21', '19-128', 'Donald Boecke']
['NaT', '19-GC-59', 'Timothy Reppucci']
['2019-11-27', '19-123', 'Timothy Reppucci']




In [25]:
#record of no AAG
record_No_AAG = download_sheet(sheet_id,"Record_of_All_Dockets_No_AAG")
record_No_AAG['Date_Filed'] = pd.to_datetime(record_No_AAG['Date_Filed'])

#display(Cases_Without_AAG)
#display(record_No_AAG)

No_AAG_uploader = pd.concat([record_No_AAG, Cases_Without_AAG])
#display(No_AAG_uploader)

No_AAG_uploader.drop_duplicates(subset = 'Docket', keep = 'first', inplace = True)

#display(No_AAG_uploader)

empty_sheet(sheet_id,"Record_of_All_Dockets_No_AAG")
add_rows(sheet_id,"Record_of_All_Dockets_No_AAG",No_AAG_uploader)


Adding...

['2019-11-27', 'NSTAR Gas Company d/b/a Eversource Energy', '19-123', 'Initial Filing', 'Petition of NSTAR Electric Company and NSTAR Gas Company each d/b/a Eversource Energy, for approval of their annual Pension/PBOP Adjustment Factors for effect January 1, 2020. Notice of Appearance of Counsel for Steven Frias. Direct Testimony of Douglas P. Horton and Timothy J. Griffin. ']
['2019-11-26', 'Massachusetts Electric Company and Nantucket Electric Company, each d/b/a National Grid', '19-144', 'Initial Filing', '2020 Vegetation Management Factor Filing. Testimony and Exhibits of Ryan A. Moe and Mindy Rosen. Motion for Exception from Department Practice. Motion for Protective Treatment of Confidential Information.  Notice of Appearance of Counsel for Meabh Purcell. ']
['2019-11-25', 'Fitchburg Gas and Electric Light Company d/b/a Unitil', '19-cmi-02', 'Initial Filing', 'Tables 1 and 2 containing customer migration data for Fitchburg Gas and Electric Light Company d/b/a Unitil. 

In [32]:
#Weekly No AAGs
weekly_No_AAG = download_sheet(sheet_id,"New_Weekly_Dockets_Without_AAGs")


empty_sheet(sheet_id,"New_Weekly_Dockets_Without_AAGs")
add_rows(sheet_id,"New_Weekly_Dockets_Without_AAGs",Cases_Without_AAG)


Adding...





In [27]:
display(Cases_Without_AAG)

,Date_Filed,Filer,Docket,File Type,Description


In [28]:
#Weekly Briefs Weekly_New_AGO_Briefs
weekly_AGO_Briefs = download_sheet(sheet_id,"Weekly_New_AGO_Briefs")

empty_sheet(sheet_id,"Weekly_New_AGO_Briefs")
#add_rows(sheet_id,"Weekly_New_AGO_Briefs",New_Briefs)


In [29]:
#Weekly Orders
weekly_Orders = download_sheet(sheet_id,"Weekly_New_Orders")

empty_sheet(sheet_id,"Weekly_New_Orders")
#add_rows(sheet_id,"Weekly_New_Orders",All_New_Orders)

In [30]:
#Weekly AGO Order
weekly_AGO_Orders = download_sheet(sheet_id,"Weekly_New_Orders_With_AAGs")

empty_sheet(sheet_id,"Weekly_New_Orders_With_AAGs")
#add_rows(sheet_id,"Weekly_New_Orders_With_AAGs",Orders_with_AAGs)

In [31]:
print('Ahhhh All Done. Time for a Beer. Glug Glug Glug')

Ahhhh All Done. Time for a Beer. Glug Glug Glug
